# Imports

In [ ]:
import pandas as pd
import plotly.express as px
import reverse_geocoder as rg
import io
import geopandas as gpd
import datetime

# Traitement des données

In [ ]:
france: pd.DataFrame = pd.read_csv("datas/caracteristics.csv", dtype={"long": str})

france.dropna(subset=["lat", "long"], inplace=True)

# Restrict our datas to metropolitan France
france = france[france.lat != 0]
france = france[france.long != 0]
france = france[france.long != "-"]
france = france[france.gps == "M"]

# Convert the coordinates into readable ones
france.long = pd.to_numeric(france.long)
france.long = france.long / 100000
france.lat = france.lat / 100000

france.an += 2000

# Create a tuple with the coordinates lat | long => (lat, long)
france["coordinates"] = france[["lat", "long"]].apply(tuple, axis=1)

def extract_time(x):
    """
    Convert the string date from the caracteristics dataset into a datetime instance
    """
    year = x[0]
    month = x[1]
    day = x[2]
    time = str(x[3])
    time = time.zfill(4)

    hour = str(time)[:1]
    minutes = str(time)[2:]

    return datetime.datetime(year, month, day, int(hour), int(minutes))
france["datetime"] = france[["an", "mois", "jour", "hrmn"]].apply(extract_time, axis=1)

def parse_lighting_conditions(x):
    """
    Convert the int lighting conditions from the caracteristics dataset into a nominal variable
    """
    if x == 1:
        return "Full day"
    elif x == 2:
        return "Twilight or dawn"
    elif x == 3:
        return "Night without public lighting"
    elif x == 4:
        return "Night with public lighting not lit"
    elif x == 5:
        return "Night with public lighting on"
france["lum_str"] = france.lum.apply(parse_lighting_conditions)

# Sort the dataframe by dates
france = france.sort_values(["datetime"])
france.reset_index(inplace=True)

france

# Validation de la position d'un accident

In [ ]:
# Verify that all positions are in the France territory
france_informations = pd.read_csv("datas/rg_france.csv")
france_informations[["lat", "lon", "name", "admin1", "admin2", "cc"]].to_csv("datas/rg_france_min.csv", index=False)
france_informations

## Quelles sont les lignes du dataset n'étant pas sur le terrain Français
L'output de cette opération renvoi un dataframe vide, cela signifie que tout nos points, même ceux qui ont des coordonées invalides (accidents d'étant passé dans la mer/océan) sont concidéré comme s'étant produit en France. Après avoir utilisé plusieurs méthode afin de retirer ces points, il est manifestement pas possible de le faire simplement. On a alors laissé ce code de validation dans le notebook afin d'avoir une certaine validation des points. Cependant ces accidents ne représentent qu'un minuscule pourcentage du total, il n'est donc pas réellement dérangeant de les concidéré pour nos statistiques, car ils n'auront pratiquement aucun impact.

In [ ]:
# Retreive all the rows that are not in France

geo = rg.RGeocoder(
    mode=2,
    verbose=True,
    stream=io.StringIO(open("datas/rg_france_min.csv", encoding="utf-8").read())
)

countries = pd.DataFrame(geo.query(france.coordinates.to_list()))
countries.drop(["lat", "lon"], axis=1, inplace=True)

france = pd.concat([france, countries], axis=1)
france[france.cc != "FR"]

# Sauvegarde des caractéristiques post-process

In [ ]:
france.to_csv("datas/caracteristics_complete.csv")

# Carte de densités des accidents
La carte de densité permet d'identifier les zone les plus denses en accidents, ici l'on peut voir un analysant la carte que les zone les plus dense sous bel et bien les grande ville comme Paris. Et si l'on zoom suffisament on peut identifier que les zone étant les plus propices aux accidents sont les intersections.

In [ ]:
def density_map(df: pd.DataFrame, params: dict = dict()):
    """
    Create a density map (heat map)
    """
    density_params = dict(
        lat="lat",
        lon="long",
        radius=5,
        mapbox_style="open-street-map",
        zoom=5,
        height=1000
    )

    fig = px.density_mapbox(
        df,
        **density_params,
        **params
    )

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.show(renderer="notebook")

In [ ]:
density_map(france)

# Carte de tous les accidents
Cette carte permet d'identifier individuellement les accidents de notre dataframe, grâce à cette carte on peut identifier les anomalies où les accidents ne se sont pas dérouler en France mais qui sont pourtant dans notre jeu de données.

In [ ]:
def scatter_map(df: pd.DataFrame, params: dict = dict()):
    """
    Create a scatter map (each dot represent an object)
    """
    scatter_params = dict(
        lat="lat",
        lon="long",
        hover_data=["an", "mois", "jour", "adr", "dep", "Num_Acc", "cc", "name", "admin1", "admin2"],
        color_discrete_sequence=["fuchsia"],
        mapbox_style="open-street-map",
        zoom=5,
        height=1000
    )

    fig = px.scatter_mapbox(
        df,
        **scatter_params,
        **params
    )

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.show(renderer="notebook")

In [ ]:
scatter_map(france)

# Carte des départements
Cette représentation est intéressant afin d'isoler les densité d'une certaine quantité par départements. Ici l'on affiche la densité d'accident par département, qui est représenter par une couleur allant du bleu au jaune, respectivement pour une petite et grande densité d'accidents.

In [ ]:
def dep_map(df, params=dict()):
    """
    Create a France departement map, with a color that represents a density
    """
    fr = gpd.read_file("datas/departements.geojson")

    choropleth_params = dict(
        geojson=fr,
        locations="dep",
        color="count",
        mapbox_style="carto-positron",
        featureidkey="properties.dep",
        zoom=5,
        center={"lat": 46, "lon": 2},
        opacity=0.5,
        labels={"value": "Nombre d'accidents"},
        height=1000
    )

    fig = px.choropleth_mapbox(
        df,
        **choropleth_params,
        **params
    )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show(renderer="notebook")

In [ ]:
dep_count = france.dep.value_counts().reset_index()
dep_count.columns = ["dep", "count"]
dep_count.dep /= 10

dep_map(dep_count)

# Fonction créatrices des graphiques d'évolution
Il serait maintenant intéressant de représenter une évolution de ces carte en fonction d'un paramètre, les fonctions suivantes génèrent une map munit d'un slider afin de connaître cette évolution.

In [ ]:
def dep_evolution(field: str):
    """
    Create a France departement map, with a variable value that depends on a field
    """
    df_accident_month_dep = france.groupby([field, "dep"]).count()["Num_Acc"].reset_index()
    df_accident_month_dep.columns = [field, "dep", "count"]
    df_accident_month_dep.dep /= 10
    
    dep_map(
        df_accident_month_dep,
        dict(animation_frame=field)
    )

def scatter_evolution(field: str):
    """
    Create scatter map, with a variable value that depends on a field
    """
    scatter_map(france, dict(animation_frame=field))

def density_evolution(field: str):
    """
    Create density map, with a variable value that depends on a field
    """
    density_map(france, dict(animation_frame=field))

def pie(df: pd.DataFrame, column: str, title: str = None):
    """
    Create a pie chart from a specific column of a dataframe
    """
    data = df[column].value_counts() / len(df)
    
    fig = px.pie(
        data,
        title=column if title == None else title,
        values=data,
        names=data.index
    )

    fig.update_traces(textposition="inside", textinfo="percent+label")

    return fig.show()

# Evolution de la carte en fonction des mois
Il est important d'également connaître la répartition des accidents pour chacun des mois, afin de bien interpréter certaines graphique de map, comme la représentation par département qui nous permet pas de connaitre la répartition des accidents par mois. Cela est moins le cas pour une map de type "scatter" car on identifi la répartition des mois en comptant le nombre de points (ce qui se fait de manière plutôt implicite dans notre cerveau)

In [ ]:
pie(france, "mois", "Répartition des mois")

In [ ]:
dep_evolution("mois")

In [ ]:
scatter_evolution("mois")

In [ ]:
density_evolution("mois")

# Evolution de la carte durant les années

In [ ]:
pie(france, "an", "Répartition des années")

In [ ]:
dep_evolution("an")

In [ ]:
scatter_evolution("an")

In [ ]:
density_evolution("an")

# Carte en fonction des conditions lumineuses

In [ ]:
pie(france, "lum_str", "Répartition des conditions lumineuses")

In [ ]:
density_evolution("lum_str")

In [ ]:
scatter_evolution("lum_str")